<a href="https://colab.research.google.com/github/mbsuraj/Coursera_Capstone/blob/master/SegmentingAndClusteringTorontoNeighborhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd


In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_list = pd.read_html(wiki_url)
print(toronto_list)

[    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns],                                                   0   ...   17
0                                                NaN  ...  NaN
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...  ...  NaN
2                                                 NL  ...   YT
3                                                  A  ...    Y

[4 rows x 

**Lets check the data  and how it looks like.**

In [3]:
toronto_df = toronto_list[0]
toronto_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
print(toronto_df.head())
print(toronto_df.shape)
print(toronto_df.columns)

  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
(287, 3)
Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


**Removing entries with no Borough names.**

In [4]:
#preprocessing
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [5]:
# sorting the rows for ease of coding.
toronto_df.sort_values('PostalCode', axis = 0, inplace = True)
toronto_df.reset_index(inplace = True, drop = True)
toronto_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek


In [6]:
toronto_df[toronto_df['PostalCode']=='M1B']['Neighborhood']

0      Rouge
1    Malvern
Name: Neighborhood, dtype: object

**Modifying Neighborhood column such that the ones with same borough are displayed together.**

In [7]:
PostalCodes = toronto_df['PostalCode'].unique()
 
toronto_df[toronto_df['PostalCode']=='M1B']['Neighborhood']
new_dict = {}
for code in PostalCodes:
  lst = []
  condition = toronto_df['PostalCode'] == code
  common_neighborhoods = list(toronto_df[condition]['Neighborhood'])
  lst.append(', '.join(common_neighborhoods))
  new_dict[code] = lst[0]
new_dict


{'M1B': 'Rouge, Malvern',
 'M1C': 'Port Union, Rouge Hill, Highland Creek',
 'M1E': 'Guildwood, Morningside, West Hill',
 'M1G': 'Woburn',
 'M1H': 'Cedarbrae',
 'M1J': 'Scarborough Village',
 'M1K': 'East Birchmount Park, Ionview, Kennedy Park',
 'M1L': 'Golden Mile, Oakridge, Clairlea',
 'M1M': 'Cliffcrest, Scarborough Village West, Cliffside',
 'M1N': 'Cliffside West, Birch Cliff',
 'M1P': 'Wexford Heights, Dorset Park, Scarborough Town Centre',
 'M1R': 'Maryvale, Wexford',
 'M1S': 'Agincourt',
 'M1T': "Sullivan, Clarks Corners, Tam O'Shanter",
 'M1V': "Steeles East, Milliken, L'Amoreaux East, Agincourt North",
 'M1W': "L'Amoreaux West",
 'M1X': 'Upper Rouge',
 'M2H': 'Hillcrest Village',
 'M2J': 'Henry Farm, Fairview, Oriole',
 'M2K': 'Bayview Village',
 'M2L': 'Silver Hills, York Mills',
 'M2M': 'Willowdale, Newtonbrook',
 'M2N': 'Willowdale South',
 'M2P': 'York Mills West',
 'M2R': 'Willowdale West',
 'M3A': 'Parkwoods',
 'M3B': 'Don Mills North',
 'M3C': 'Don Mills South, Flemin

In [8]:
neighborhoods_df = pd.DataFrame(new_dict.items(), columns = ['PostalCode', 'Neighborhoods'])
toronto_df = pd.merge(toronto_df, neighborhoods_df, how = 'outer', on = 'PostalCode')
toronto_df.drop('Neighborhood', axis = 1, inplace =True)
toronto_df.rename(columns = {'Neighborhoods':'Neighborhood'}, inplace = True)
toronto_df.drop_duplicates(inplace = True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
5,M1E,Scarborough,"Guildwood, Morningside, West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [9]:
#checking
toronto_df[toronto_df['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood
96,M5G,Downtown Toronto,Central Bay Street


**Replacing Not assigned neighborhood with corresponing Borough**

In [10]:
condition = toronto_df['Neighborhood'] == 'Not assigned'
for i in toronto_df.index:
  if toronto_df['Neighborhood'][i] == 'Not assigned':
    toronto_df['Neighborhood'][i] = toronto_df.loc[i]['Borough']
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
5,M1E,Scarborough,"Guildwood, Morningside, West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [11]:
#check
condition = toronto_df['Borough'] == 'Queen\'s Park'
toronto_df[condition]

,PostalCode,Borough,Neighborhood
182,M9A,Queen's Park,Queen's Park


In [14]:
toronto_df.shape

(103, 3)